In [ ]:
!pip install gurobipy
!pip install tqdm

In [ ]:
from collections import defaultdict
from tqdm import tqdm
import gurobipy as gp
import json
from enum import Enum
import pandas as pd

In [ ]:
class VariableType(Enum):
    FLOW = 1
    STOCK = 2
    CELL = 3

class CostType(Enum):
    MISSING_INPUT = 1
    MISSING_MINIMUM_QUANTITIES = 2
    MISSING_OUTPUT = 3
    LOGISTIC = 4


In [ ]:
folderToData = "/content/"
path = f"{folderToData}/data.json"

In [ ]:
with open(path, "rb")  as fp :
    data = json.load(fp)

In [ ]:
def addHeapStockVariable(model, variables,  nodeId: str):
    if nodeId in data["intermediateNodes"]:
        maxCapacity = sum(data["intermediateNodes"][nodeId]["cells"].values())
        if  nodeId not in variables[VariableType.STOCK.value]:
            variables[VariableType.STOCK.value][nodeId] = model.addVar(0,maxCapacity)


def computeMissingInputOutput(model,variables,objectives, negativeContributions, positiveContributions):
    for nodeId, value in data["terminalNodes"].items():
        if value < 0 :
            flows = gp.quicksum([variables[VariableType.FLOW.value][idx]*ratio for idx,ratio in positiveContributions[nodeId]])
            quantity = -value
            model.addLConstr(
                lhs = flows,
                rhs = quantity,
                sense = gp.GRB.LESS_EQUAL
            )
            objectives[CostType.MISSING_OUTPUT] += (quantity - flows)

        else:
            flows = gp.quicksum([variables[VariableType.FLOW.value][idx]*ratio for idx,ratio in negativeContributions[nodeId]])
            quantity = value
            model.addLConstr(
                lhs = flows,
                rhs = quantity,
                sense = gp.GRB.LESS_EQUAL
            )
            objectives[CostType.MISSING_INPUT] += (quantity - flows)

def computeMissingMinimumQuantitiesToReach(model,variables, limitQuantities,objectives, costType):
    for details in limitQuantities.values():
        l_edges, minimumQuantity, maximumQuantity = details[0], details[1], details[2]
        if len(l_edges) == 0:
            continue
        flows = gp.quicksum([variables[VariableType.FLOW.value][idx]*ratio for idx,ratio in l_edges])
        if maximumQuantity is not None:
            model.addLConstr(
                lhs = flows,
                rhs = maximumQuantity,
                sense = gp.GRB.LESS_EQUAL
            )
        if minimumQuantity>0:
            slackVariable = model.addVar(lb = -float("inf"))
            model.addLConstr(
                lhs = flows + slackVariable,
                rhs = minimumQuantity,
                sense = gp.GRB.GREATER_EQUAL
            )

            objectives[costType] += slackVariable


In [ ]:
objectives = {
value: gp.LinExpr() for value in CostType.__members__.values()
}

variables = defaultdict(dict)
variables[VariableType.CELL.value] = defaultdict(dict)

positiveContributions = defaultdict(list)
negativeContributions = defaultdict(list)

model = gp.Model("HeapModel")

for edgeId, edge in tqdm(data["edges"].items()):


    from_,to_,transport = edge["from"],edge["to"],edge["transport"]

    addHeapStockVariable(model, variables,from_)
    addHeapStockVariable(model, variables, to_)

    positiveContributions[to_].append((edgeId,1))
    negativeContributions[from_].append((edgeId,1))

    if to_ in data["transformedNodes"]:
        for transformedNode, ratio in data["transformedNodes"][to_][from_].items():
            positiveContributions[transformedNode].append((edgeId, ratio))

    variables[VariableType.FLOW.value][edgeId] = model.addVar()
    if transport is not None:
        cost = edge["cost"]
        objectives[CostType.LOGISTIC] += (cost * variables[VariableType.FLOW.value][edgeId])



for heapNodeId, heapNode in tqdm(data["intermediateNodes"].items()):
    stockVariable = variables[VariableType.STOCK.value].get(heapNodeId)
    timePeriodId = heapNode["timePeriod"]
    if stockVariable is None:
        continue
    mixingCells = gp.LinExpr()
    for cellId, capacity in heapNode["cells"].items():
        binaryVar = model.addVar(vtype=gp.GRB.BINARY)
        variables[VariableType.CELL.value][(cellId,timePeriodId)][heapNodeId] = binaryVar
        mixingCells += (binaryVar*capacity)


    comingFlows = gp.quicksum([variables[VariableType.FLOW.value][idx]*ratio for idx, ratio in positiveContributions[heapNodeId]])
    exitingFlows = gp.quicksum([variables[VariableType.FLOW.value][idx]*ratio for idx, ratio in negativeContributions[heapNodeId]])
    previousStock = gp.quicksum([variables[VariableType.STOCK.value].get(heapNode["previous"],gp.LinExpr())])

    variable = variables[VariableType.STOCK.value][heapNodeId]
    model.addLConstr(
        lhs = variable,
        rhs = previousStock + comingFlows - exitingFlows,
        sense = gp.GRB.EQUAL
    )
    model.addLConstr(
        lhs = variable,
        rhs = mixingCells,
        sense = gp.GRB.LESS_EQUAL)

for cell, cellsVariables in variables[VariableType.CELL.value].items():
    cellId, timePeriodId = cell
    cellsVariables = list(cellsVariables.values())
    model.addLConstr(
        lhs = gp.quicksum(cellsVariables),
        rhs = 1,
        sense = gp.GRB.LESS_EQUAL
    )

computeMissingMinimumQuantitiesToReach(model, variables, data["limitQuantities"], objectives, CostType.MISSING_MINIMUM_QUANTITIES)
computeMissingInputOutput(model,variables,objectives, negativeContributions, positiveContributions)

model.setObjectiveN(
    objectives[CostType.MISSING_MINIMUM_QUANTITIES],
    index=0,
    priority=len(objectives),
    weight=1,
)
model.setObjectiveN(
    objectives[CostType.MISSING_INPUT],
    index=1,
    priority=len(objectives)-1,
    weight=1,
)

model.setObjectiveN(
    objectives[CostType.MISSING_OUTPUT],
    index=2,
    priority=len(objectives)-2,
    weight=1,
)
model.setObjectiveN(
    objectives[CostType.LOGISTIC],
    index=3,
    priority=len(objectives)-3,
    weight=1,
)
model.setParam("MIPGap", 0.01)
model.optimize()

Restricted license - for non-production use only - expires 2026-11-23


100%|██████████| 100/100 [00:00<00:00, 7852.88it/s]

Set parameter MIPGap to value 0.01
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
MIPGap  0.01

Optimize a model with 282 rows, 1007 columns and 2099 nonzeros
Model fingerprint: 0x95269f89
Variable types: 527 continuous, 480 integer (480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 5e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve...
---------------------------------------------------------------------------

Presolve removed 15 rows and 4 columns
Presolve time: 0.00s

In [ ]:
def writeSolution(variables):
    flows = []
    cells = []
    for edgeId, var in variables[VariableType.FLOW.value].items():
        if var.X > 1e-3:
            flows.append(
                {
                    "edgeId": edgeId,
                    "value": var.X
                }
            )
    for cellIdPeriod, productDict in variables[VariableType.CELL.value].items():
        for productId, var in productDict.items():
            if var.X > 1e-3:
                cells.append(
                    {
                        "cellId": cellIdPeriod[0],
                        "timePeriodId": cellIdPeriod[1],
                        "productId": productId,
                        "value": var.X
                    }
                )
    return pd.DataFrame(flows), pd.DataFrame(cells)

In [ ]:
flows, cells = writeSolution(variables)

In [ ]:
flows.to_csv(f"{folderToData}/flows.csv", index=False)
cells.to_csv(f"{folderToData}/cells.csv", index=False)